# 4 - INTERACIÓN VERTICAL

Este script en un ejemplo de una herramienta de inteligencia artificial que coordina la ejecución de varios agentes (AI) especializados para resolver una tarea en común, en este caso, "Crear un juego funcional de SNAKE".

## 4.1 - Importaciones y configuraciones

In [ ]:
import os
import logging
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
from colorama import Fore
from exec_code import execute_code, extract_code_from_string
from prompt import CODE_DEVELOPER_SYSTEMPROMPT, COORDINATOR_SYSTEMPROMPT, CRITIC_ROLE_PROMPT, SOLVER_ROLE_PROMPT, TESTER_SYSTEMPROMPT, UIDESIGN_SYSTEMPROMPT, EXPERT_USER_EXPERIENCE_SYSTEMPROMPT
load_dotenv()

OPENAI_DEFAULT_MODEL=os.getenv("OPENAI_DEFAULT_MODEL")

client = OpenAI()

logger = logging.getLogger(__name__)

## 4.2- Declaracion de tipos de agentes

Clase `Agent`:
- `Agent` es una clase abstracta base (ABC) que define una estructura para otros agentes.
- Tiene un constructor que acepta un modelo de la API de OpenAI y un mensaje de sistema inicial.
- `add_to_memory` almacena los mensajes en memoria para futuras referencias.
- `get_schema` devuelve el esquema de entrada/salida del agente
- `run` es un método abstracto que debe ser implementado por las subclases para tomar una entrada del usuario y ejecutar el comportamiento del agente.

Clase `ConversationAgent`:
- Es un agente derivado de `Agent` orientado a conversaciones.
- Su `run` método captura la entrada del usuario, la añade a la memoria y utiliza la API de ChatGPT para generar una respuesta.

Clase `CodeExecuterAgent`:
- Es otro agente derivado de `Agent` que se especializa en ejecutar código.
- Su método `run` intenta extraer y ejecutar código de la entrada del usuario.
- Utiliza funciones `execute_code` y `extract_code_from_string` para manejar la ejecución del código.
- También captura la respuesta generada y se comunica con OpenAI ChatGPT para obtener retroalimentación.


In [ ]:
class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        function: Optional[dict] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        self.function=function
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        suffix_prompt: Optional[str] = None,
        agent_name: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        

        self.agent_name = agent_name
        self.description = description
        
        if suffix_prompt:
            self.suffix_prompt=[{"role": "user", "content": suffix_prompt}]
        else:
            self.suffix_prompt=[]

        function={
            "name": self.agent_name,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
        
        super().__init__(model,system_prompt,function)
    
    def run(self,prompt):
        self.add_to_memory("user", prompt)
        
        #print(f"self.memory:{self.memory}\n\n")
        
        print(f"messages:{self.memory  + self.suffix_prompt}\n\n")
        
        completion=''
        stream = client.chat.completions.create(
            messages=self.memory  + self.suffix_prompt,
            model=self.model,
            stream=True
        )

        for chunk in stream:
            text_chunk=chunk.choices[0].delta.content
            if text_chunk:
                completion+=text_chunk
                print(text_chunk, end='', flush=True)
        
        self.add_to_memory("assistant", completion)
        
        return completion

class CodeExecuterAgent(Agent):

    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        suffix_prompt: Optional[str] = None,
        agent_name: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:

        self.agent_name = agent_name
        self.description = description

        if suffix_prompt:
            self.suffix_prompt=[{"role": "user", "content": suffix_prompt}]
        else:
            self.suffix_prompt=[]
                
        function={
            "name": self.agent_name,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
        
        super().__init__(model,system_prompt,function)
    
    def run(self,prompt):
        
        completion=''
        code_exec_result=''
        
        
        # Execute code if found
        ######################################
        code_to_execute = extract_code_from_string(prompt)
        if code_to_execute:
            try:
                print("Executing code...")
                exec_response = execute_code(code_to_execute,use_docker=False)
                if exec_response[0] == 0:
                    print(f"Code executed successfully")
                    code_exec_result=f'{prompt}\n\nCode executed successfully'
                else:
                    print(f"Error executing code: {exec_response[1]}")
                    code_exec_result=f'{prompt}\n\nError executing code: {exec_response[1]}'
            except Exception as e:
                print(f"Error executing code: {e}")
                code_exec_result=f'{prompt}\n\nError executing code: {e}'
        else:
            print(f"No code found to execute.")
            prompt+=f'{prompt}\n\nError executing code: bad format'
        
        prompt+=code_exec_result
        
        self.add_to_memory("user", prompt)
        
        #print(f"self.memory:{self.memory}\n\n")
        
        print(f"messages:{self.memory  + self.suffix_prompt}\n\n")
    
        # Review code
        ######################################
        stream = client.chat.completions.create(
            messages=self.memory  + self.suffix_prompt,
            model=self.model,
            stream=True
        )

        for chunk in stream:
            text_chunk=chunk.choices[0].delta.content
            if text_chunk:
                completion+=text_chunk
                print(text_chunk, end='', flush=True)
        
        self.add_to_memory("assistant", completion)
        
        return completion

## 4.3- AgentManager

El fragmento define una clase `AgentManager` que gestiona la interacción entre varios agentes de inteligencia artificial en una tarea común. Específicamente, esta clase realiza lo siguiente:

El `AgentManager` define dos tipos de rol `solver` encargado de crear codigo y la proponer la solucion principal y `reviewers` encargados de criticar la solucion, generar sugerencias y nuevas caracteristicas. Utilizando el mensaje del usuario como entrada, determina el agente `solver` y todos los demas agentes alistados se convierten en `reviewers`.

- Se identifica y activa el agente `solver` correspondiente, y se guardan los otros agentes como `reviewers`.
- Hay un ciclo que permite a cada agente revisor proporcionar retroalimentación sobre la solución propuesta hasta que se alcanza un consenso o se supera el número máximo de iteraciones.
- En el ciclo, si todos los agentes están de acuerdo con la solución propuesta, se termina el proceso y se devuelve la solución.
- Si no hay consenso, la interacción entre el agente solucionador y los agentes revisores continúa, iterando sobre las propuestas.
- Una vez completado el proceso, se imprime la solución final y se agrega a la memoria del entorno.

En resumen, `AgentManager` es esencialmente un coordinador que facilita un proceso iterativo con agentes inteligentes que pueden proporcionar, revisar y perfeccionar una solución basada en colaboración mutua de tipo vertical.

In [ ]:
class AgentManager(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        list_agents: Optional[list] = None,
        max_iteration: Optional[int] = 10
    ) -> None:
        super().__init__(model,system_prompt)
        
        self.list_agents=list_agents
        self.max_iteration=max_iteration
    
    def get_agent_list_schemas(self):
        return [agent.get_schema() for agent in self.list_agents]   
    
    def run(self,user_prompt):
        
        print(Fore.MAGENTA,f"----------------- RUN MANAGER -----------------")
        print(f"Manager: I'm thinking the best agent to solve the user query -> {user_prompt}")
        
        # Choose what agent i have to execute to resolve the problem
        ######################################
        messages = self.memory + [{"role": "user", "content": f"[PROMPT]\n{user_prompt}\n\n[AGENTS]\n{self.get_agent_list_schemas()}"}]
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=self.model,
            functions=self.get_agent_list_schemas()
        )
        
        # Get the agent to execute
        solver_agent_name=chat_completion.choices[0].message.function_call.name
        
        # Manual choose the solver agent to execute
        # solver_agent_name=self.list_agents[0].agent_name
        
        print(f"Manager: the best agent to solve this is -> {solver_agent_name}")        

        # Get the agent class to use like main solver
        solver_agent_class = next((agent for agent in self.list_agents if agent.agent_name == solver_agent_name), None)

        # Create a new list with all agent without the solver agent
        ######################################
        reviewers_agents = self.list_agents.copy()
        reviewers_agents.remove(solver_agent_class)      
        
        # Initialize variables for loop
        main_proposal = user_prompt
        feedback=''
        base_colors = [Fore.CYAN, Fore.YELLOW, Fore.BLUE]
        
        # Steps to execute
        for i in range(self.max_iteration):
            
            print(Fore.MAGENTA,f"\n\n-----------------> ITERATION: {i+1}")
            
            # press enter to continue
            #input("Press Enter to continue...")
            
            agree_agents = 0
            for index, reviewer_agent in enumerate(reviewers_agents):
                
                # Check agent feedback
                if "Code is fine" in feedback:
                    agree_agents += 1
                
                # execute the solver agent
                else:
                    print(Fore.GREEN, f"\n\nAgent Solver -> {solver_agent_class.agent_name} is proposing a solution")

                    if i == 0 and index == 0:
                        # Add to manager/enviroment memory
                        self.add_to_memory("user", user_prompt)
                        # Add to agents memoty
                        for agent in reviewers_agents:
                            agent.add_to_memory("user", user_prompt)

                        main_proposal = solver_agent_class.run(user_prompt)
                        
                        # Add to manager/enviroment memory
                        self.add_to_memory("user", f"[DEVELOPER_PROPOSAL]:{main_proposal}")
                        
                    else:
                        main_proposal = solver_agent_class.run(f"[{reviewer_agent.agent_name}]: {feedback}")
                    agree_agents = 0
                        
                # Colores base de Colorama
                color = base_colors[index % len(base_colors)]
                
                print(color,f"\n\nFeedback from: {reviewer_agent.agent_name}")

                # Execute the reviewers agent
                feedback = reviewer_agent.run(f"[DEVELOPER_PROPOSAL]:{main_proposal}")
                
                # Add to manager/enviroment memory
                self.add_to_memory("user", f"[{reviewer_agent.agent_name}]: {feedback}")

                print(f'\n\nagree_agents: {agree_agents}/{len(reviewers_agents)}')
            
            # Check if all agents agree with the code solution
            if agree_agents >= len(reviewers_agents):
                final_answer=f"Manager: Consensus reached with all agents!!!\n"
                break
        
        # End with the final answer
        
        print(Fore.MAGENTA,f"Manager: Internal Memory of enviroment: \n\n{self.memory}\n\n")
        
        final_answer=f"Manager: the final solution is -> \n{main_proposal}"
        print(final_answer)
        
        # Add to manager/enviroment memory
        self.add_to_memory("assistant", final_answer)
        
        return main_proposal

## 4.4- Configuracion del AgentManager

Cada agente hereda un tipo de agentes `ConversationAgent` y `CodeExecuterAgent`, para generar conversaciones criticas y ejecucion de codigo, respectivamente. El script define cuatro agentes específicos:

1. `GameCodeDeveloper`: encargado de desarrollar código para juegos en Python. ()
2. `UIDesignDeveloper`: enfocado en resolver problemas de diseño de interfaz de usuario en Python.
3. `TesterDeveloper`: destinado a ejecutar y probar código en Python.
4. `ExperienceUserExpert`: dedicado a planificar la lógica del juego para la experiencia del usuario en Python.

Todos estos agentes están diseñados para trabajar como parte de un sistema más grande manejado por `AgentManager`, que determina qué agente es el mejor para resolver la consulta del usuario y coordina revisores para iterar hacia una solución final.

In [ ]:
GameCodeDeveloper=ConversationAgent(
    model=OPENAI_DEFAULT_MODEL,
    agent_name="GameCodeDeveloper",
    description="Agent to developer code games in python",
    system_prompt=CODE_DEVELOPER_SYSTEMPROMPT,
    suffix_prompt=SOLVER_ROLE_PROMPT
)

UIDesignDeveloper=ConversationAgent(
    model=OPENAI_DEFAULT_MODEL,
    agent_name="UIDesignDeveloper",
    description="Agent to resolve UI design problems in python",
    system_prompt=UIDESIGN_SYSTEMPROMPT,
    suffix_prompt=CRITIC_ROLE_PROMPT
)

TesterDeveloper=CodeExecuterAgent(
    model=OPENAI_DEFAULT_MODEL,
    agent_name="TesterDeveloper",
    description="Agent to execute and test code in python",
    system_prompt=TESTER_SYSTEMPROMPT,
    suffix_prompt=CRITIC_ROLE_PROMPT
)

manager=AgentManager(
    model=OPENAI_DEFAULT_MODEL,
    list_agents=[
        GameCodeDeveloper,   
        UIDesignDeveloper,   
        TesterDeveloper        
    ],
    system_prompt=COORDINATOR_SYSTEMPROMPT,
    max_iteration=5
)

## 4.5- Ejecutar!

In [ ]:
manager.run("Create a functional snake game")